# Gridsearch
## N-BEATS

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import shutil
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook as tqdm

from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel, NBEATSModel
from darts.metrics import mape
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, SunspotsDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.timeseries_generation import holidays_timeseries
import datetime

import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)


[2021-10-23 00:08:32,931] WARNING | darts.models | Support Facebook Prophet is not available. To enable it, install u8darts[prophet] or u8darts[all].


In [3]:
data_root = os.path.join('..', '..', 'RDN', 'Load Data (2018-2019)', 'artifacts')
first_val_day = pd.Timestamp('20191201')
last_train_day = pd.Timestamp('20191130')


In [4]:
# load dataset
series = TimeSeries.from_series(pd.read_csv(os.path.join(data_root, 'load_15min.csv'), index_col=0, parse_dates=True), freq='15min')

# Train / Test split
train, val = series.split_before(first_val_day)

# Normalize the time series (note: we avoid fitting the transformer on the validation set)
transformer = Scaler()
train_transformed = transformer.fit_transform(train)
val_transformed = transformer.transform(val)
series_transformed = transformer.transform(series)

# create month and year covariate series
year_60 = datetime_attribute_timeseries(time_index=series_transformed, attribute='year')
month_60 = datetime_attribute_timeseries(time_index=series_transformed, attribute='month', cyclic=True)
day_60 = datetime_attribute_timeseries(time_index=series_transformed, attribute='day', cyclic=True)
hour_60 = datetime_attribute_timeseries(time_index=series_transformed, attribute='hour', cyclic=True)
dayofweek_60 = datetime_attribute_timeseries(time_index=series_transformed, attribute='dayofweek')
dayofyear_60 = datetime_attribute_timeseries(time_index=series_transformed, attribute='dayofyear')
weekofyear_60 = datetime_attribute_timeseries(time_index=series_transformed, attribute='weekofyear')
holidays_60 = holidays_timeseries(time_index=series_transformed.time_index, country_code='PT')

covariates = year_60.stack(month_60) \
                    .stack(day_60) \
                    .stack(hour_60) \
                    .stack(dayofweek_60) \
                    .stack(dayofyear_60) \
                    .stack(weekofyear_60) \
                    .stack(holidays_60)

# scale them between 0 and 1:
transformer = Scaler()
covariates = transformer.fit_transform(covariates)

# split in train/validation sets:
cov_train, cov_val = covariates.split_before(pd.Timestamp(first_val_day))


## 15 min, 1hr-ahead forecasts

In [7]:
n_epochs = 200
forecast_horizon = 4*3
input_chunk_length = 24*14
random_state = 0

nbeats_15 = NBEATSModel(input_chunk_length=4,
                        output_chunk_length=forecast_horizon,
                        random_state=random_state,
                        torch_device_str='cuda',
                        nr_epochs_val_period=30,
                        log_tensorboard=True,
                        force_reset=True,
                        model_name=f'N-beats_Gridsearch_15min_20180101_{last_train_day}_horizon_{str(int(forecast_horizon))}h_epochs_{str(n_epochs)}_random_state_{random_state}')
param_grid = {
    "input_chunk_length": [4*3, 4*24, 4*24*7, 4*24*14],
    "output_chunk_length": [4*3]
    }
nbeats_15.gridsearch(parameters=param_grid,
                     stride=1,
                     series=series_transformed,
                     forecast_horizon=forecast_horizon
                     )


RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 6.00 GiB total capacity; 370.31 MiB already allocated; 42.67 KiB free; 372.00 MiB reserved in total by PyTorch)